In [1]:
import sys
import os

sys.path.append('../src-py/')
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [2]:
import pandas as pd
import numpy as np
from tqdm import tqdm
tqdm.pandas()

pd.set_option('display.max_colwidth', None)

In [3]:
from datasets import load_metric
metric = load_metric("rouge")

In [4]:
# sentence similarity
#import spacy
#nlp=spacy.load("en_core_web_lg")

In [6]:
from scipy import sparse
from fast_pagerank import pagerank
from fast_pagerank import pagerank_power
from sklearn.metrics.pairwise import cosine_similarity

from sentence_transformers import SentenceTransformer, InputExample, LoggingHandler, losses, models, util
from transformers import BertForSequenceClassification, AutoTokenizer, TextClassificationPipeline
import torch
import json

In [7]:
from nltk.tokenize import word_tokenize

In [8]:
from claim_scoring import *

### This notebook is to train and evalaute ArgLexRank algorithm:

- Paper: https://webis.de/downloads/publications/papers/alshomary_2020b.pdf

#### We will test different versions of argumentation scorers:
- Claim identification (https://aclanthology.org/N19-1054.pdf) (DONE)
- Claim Lexicon (https://github.com/hjshah142/SnippetGeneration/blob/master/data/ClaimLexicon.txt) Count number of words that appear in the lexicon and consider it the score of the sentence.
- Argumentation quality assessment Gertz et al. (https://arxiv.org/pdf/1911.11408.pdf). Code at (/mnt/ceph/storage/data-tmp/2021/jrieskamp/jl-workspace/bert-finetuning)

#### For each version, we will evaluate the top selected two sentences against the ground-truth of the following dataset: https://github.com/webis-de/sigir20-extractive-snippet-generation-for-arguments/blob/master/data/dataset_as_json_file.json

In [9]:
json_data = json.load(open('../data/dataset_as_json_file.json'))

In [10]:
df=pd.DataFrame(json_data)

In [11]:
#Loading transformer-based model for the embedding
model = SentenceTransformer('bert-base-uncased') #roberta-large??

Some weights of the model checkpoint at /mnt/ceph/storage/data-tmp/2021//sile2804/.cache/torch/sentence_transformers/bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [12]:
#Loading the claim-identification pre-trained model
claim_miner_model_path = '/mnt/ceph/storage/data-in-progress/data-research/arguana/arg-generation/claim-mining/export.pkl'
claim_miner  = ClaimMiner(claim_miner_model_path, 'Prediction')

In [ ]:
lexicon_file = open('../data/ClaimLexicon.txt')
lexicon_contents = lexicon_file.read()
lexicon_tokens = lexicon_contents.split(', ')

In [ ]:
#Gretz et al 2019 model
# set the model with trained data
gretz_model = BertForSequenceClassification.from_pretrained('/mnt/ceph/storage/data-tmp/2021/jrieskamp/jl-workspace/bert-finetuning/argQ-trainer/checkpoint-9000', local_files_only=True, cache_dir='cache')
# set the tokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased', cache_dir='cache')

In [ ]:
#Applying ArgLexRank to generate the gist of an argument

#1. Compute sbert embedding for each sentence in the post
#2. Compute argumentativeness scores for each sentence in the post
#3. For each post, run PageRank algorithm on the graph constructed from sentences of the post (or from the post and all its comments)
#4. Take top two sentences to be the conclusion

def get_sentences_lexicon_scores(sentences):
    score_sentences = []
    for sentence in sentences:
        score = 0  # number of times a unique word from a sentence also in lexicon
        words_counted = []  # list of words that are already counted in the sentence.
        words = word_tokenize(sentence.lower())
        for word in words:
            if lexicon_tokens.__contains__(word) and not words_counted.__contains__(word):
                words_counted.append(word)
                score += 1
        score_sentences.append(score/len(words))
        
    return score_sentences

def get_argumentative_quality_scores(sentences):
    
    # set the cuda device
    if torch.cuda.device_count() >= 1:
        DEVICE = torch.cuda.current_device()
    else:
        DEVICE = -1
    
    # set the pipeline
    pipeline = TextClassificationPipeline(model=gretz_model, tokenizer=tokenizer, framework='pt', task='ArgQ')
    results = pipeline(sentences, device=DEVICE)
    sentences_scores = []
    for result_dict in results:
        sentences_scores.append(result_dict['score'])
    return sentences_scores

def get_arg_scores(sentences):
    claim_scores = claim_miner.predict(sentences)
    return [x[1].item() for x in claim_scores]

def gen_match_matrix(model, sents, min_match_score=0):
    sents_embeddings  = model.encode(sents)
    
    sim_matrix = cosine_similarity(sents_embeddings, sents_embeddings)
    super_threshold_indices = sim_matrix < min_match_score
    sim_matrix[super_threshold_indices] = 0

    return sim_matrix

def extract_gist(model, post, comments = [], min_sim_score=0, 
                 min_arg_quality_score=0, p=0.8, argumentative_scorer='claim-identifier'):
    
    all_sentences = post + comments
    
    #compute argumentativeness scores and filter out sentences with quality less htan arg_quality_score
    if argumentative_scorer   == 'claim-identifier':
        all_sentences_arg_scores = get_arg_scores(all_sentences)
    elif argumentative_scorer == 'arg-quality':
        all_sentences_arg_scores = get_argumentative_quality_scores(all_sentences)
    else:
        # represented by "lexicon_score"
        all_sentences_arg_scores = get_sentences_lexicon_scores(all_sentences)
    
    
    filtered_sentences = [x for x in zip(all_sentences, all_sentences_arg_scores) 
                          if x[1] >= min_arg_quality_score]
    
    if len(filtered_sentences) < 2:
        return ['', '']
        
    filtered_sentences_texts, filtered_sentences_arg_scores = zip(*filtered_sentences)

    sim_matrix = gen_match_matrix(model, filtered_sentences_texts, min_match_score=min_sim_score)
    pr=pagerank(sim_matrix, personalize=np.array(filtered_sentences_arg_scores), p=p)

    ranked_candidates = list(zip(filtered_sentences_texts, pr))
        
    ranked_candidates = sorted(ranked_candidates, key=lambda x: -x[1])
    
    return [x[0] for x in ranked_candidates]

In [ ]:
df = df[(df.sentences.str.len() > 2) & (df.snippet.str.len() > 1)]

In [ ]:
def evaluate_snippets_rouge(gt_snippets, gen_snippets):
    gt_snippets = [' '.join(x) for x in gt_snippets]
    gen_snippets = [' '.join(x) for x in gen_snippets]
    results = metric.compute(predictions=gen_snippets, references=gt_snippets)
    #['rouge1', 'rouge2', 'rougeL', 'rougeLsum']
    return [results["rouge1"].mid.fmeasure, results["rouge2"].mid.fmeasure, results["rougeL"].mid.fmeasure]

In [ ]:
scores = []
for arg_scorer in ['claim_identifier', 'arg-quality', 'claim-lexicon']:
    for min_arg_quality_score in np.arange(0, 1.0, 0.2):
        for min_simi_score in np.arange(0, 1.0, 0.2):
            for p in np.arange(0, 1.2, 0.2):
                generated_snippets = [extract_gist(model, argument, 
                                                   min_arg_quality_score=min_arg_quality_score,
                                                   min_sim_score=min_simi_score,
                                                   p=p, argumentative_scorer = arg_scorer)
                                for argument in df.sentences]
                rouge_scores = evaluate_snippets_rouge(df.snippet.tolist(), generated_snippets)
                scores.append([arg_scorer, min_arg_quality_score, min_simi_score, p] + rouge_scores)

/usr/local/lib/python3.6/dist-packages/scipy/sparse/linalg/dsolve/linsolve.py:145: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  SparseEfficiencyWarning)
/usr/local/lib/python3.6/dist-packages/scipy/sparse/linalg/dsolve/linsolve.py:145: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  SparseEfficiencyWarning)
/usr/local/lib/python3.6/dist-packages/scipy/sparse/linalg/dsolve/linsolve.py:145: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  SparseEfficiencyWarning)
/usr/local/lib/python3.6/dist-packages/scipy/sparse/linalg/dsolve/linsolve.py:145: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  SparseEfficiencyWarning)
/usr/local/lib/python3.6/dist-packages/scipy/sparse/linalg/dsolve/linsolve.py:145: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  SparseEfficiencyWarning)
/usr/local/lib/python3.6/dist-packages/scipy/sparse/linalg/dsolve/linsolve.py:145: SparseE

In [21]:
df_all_scores = pd.DataFrame(scores, columns = ['arg_scorer', 'min-arg-quality', 'min-similarity', 'p', 'R-1', 'R-2', 'R-L'])

In [30]:
df_all_scores.to_csv('../data/arglex-rank-fine-tuning-all-scores.csv')

In [31]:
df_all_scores.head()

,arg_scorer,min-arg-quality,min-similarity,p,R-1,R-2,R-L,Unnamed: 0
0,arg-quality,0.8,0.0,0.0,0.0,0.0,0.0,NaN
1,arg-quality,0.8,0.0,0.2,0.0,0.0,0.0,NaN
2,arg-quality,0.8,0.0,0.4,0.0,0.0,0.0,NaN
3,arg-quality,0.8,0.0,0.6,0.0,0.0,0.0,NaN
4,arg-quality,0.8,0.0,0.8,0.0,0.0,0.0,NaN


### Using the best model to generate conclusion for the development dataset:

#### We selected the model: argQ with p=20 and quality score as 0.170, 0.150 or score as 0.170 and p=40

In [53]:
valid_df = pd.read_pickle('/mnt/ceph/storage/data-in-progress/data-research/arguana/arg-generation/multi-taks-counter-argument-generation/reddit_data/conclusion_and_ca_generation/valid_conclusion_comp_remove_75sem_perc_with_targets.pkl')
unique_valid_posts = valid_df.drop_duplicates('post_id')
unique_valid_posts.head()

,post_id,split,comment_id,title,post,n_sentences,counter,num_cand_conc,masked_premises,premises_with_conclusion,conclusion_stance,conclusion_targets,bart_conclusion,arglex_rank_conclusion
295914,t3_7uek1b,val,t1_dtjoww6,Pink is just light red and doesn't deserve it's own name.,"[i understand the need for more precise naming when you're doing something artsy., what i'm focusing on is how we describe colors in everyday life., pink is just a light shade of the color red., we already have a ton of precedence for just calling something a light or dark color., when people see a light or dark blue object, they just call it light blue or dark blue., when the see a light or dark green thing, they call it light green or dark green., for some reason, when we see something that is light red we call it pink., i don't see any rhyme or reason to this distinction and i don't thi...",8,"[what's your opinion on scarlet?, brick?, salmon?, rose?, wine?]",1,"[i understand the need for more precise naming when you're doing something artsy., what i'm focusing on is how we describe colors in everyday life., pink is just a light shade of the color red., we already have a ton of precedence for just calling something a light or dark color., when people see a light or dark blue object, they just call it light blue or dark blue., when the see a light or dark green thing, they call it light green or dark green., for some reason, when we see something that is light red we call it pink., i don't see any rhyme or reason to this distinction and i don't thi...","[i understand the need for more precise naming when you're doing something artsy., what i'm focusing on is how we describe colors in everyday life., pink is just a light shade of the color red., we already have a ton of precedence for just calling something a light or dark color., when people see a light or dark blue object, they just call it light blue or dark blue., when the see a light or dark green thing, they call it light green or dark green., for some reason, when we see something that is light red we call it pink., i don't see any rhyme or reason to this distinction and i don't thi...",-0.846022,Pink,i don't think pink deserves it's own name.,i don't see any rhyme or reason to this distinction and i don't think pink deserves it's name.
295939,t3_7uj99e,val,t1_dtknvk0,Elon Musk should not be selling flamethrowers as if they were toys.,"[i just saw on the news that elon musk have sold over 20 thousand flamethrowers., they appear to be oversized blowtorches designed to be just below the limits that would require licenses in most states., in a promotional video, he jokes around with it, pretending to use it on the cameraman. on twitter, he jokes about it been useful during a zombie apocalypse., but, as far as i know, no real use for it is seriously discussed., i find this behaviour borderline negligent., if these things become even more popular, it is just a matter of time until someone gets seriously hurt., either on purpo...",14,"[people get hurt with tide pods and knives, so why would we care about this?, we can't protect every dumb person in this planet from themselves.]",1,"[i just saw on the news that elon musk have sold over 20 thousand flamethrowers., they appear to be oversized blowtorches designed to be just below the limits that would require licenses in most states., in a promotional video, he jokes around with it, pretending to use it on the cameraman. on twitter, he jokes about it been useful during a zombie apocalypse., but, as far as i know, no real use for it is seriously discussed., i find this behaviour borderline negligent., if these things become even more popular, it is just a matter of time until someone gets seriously hurt., either on purpo...","[i just saw on the news that elon musk have sold over 20 thousand flamethrowers., they appear to be oversized blowtorches designed to be just below the limits that would require licenses in most states., in a promotional video, he jok

In [56]:
unique_valid_posts['arglex_rank_conclusion'] = unique_valid_posts['masked_premises'].apply(lambda x : extract_gist(model, x, min_arg_quality_score=0.170, p=0.2, argumentative_scorer = 'argQ'))

/usr/local/lib/python3.6/dist-packages/scipy/sparse/linalg/dsolve/linsolve.py:145: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  SparseEfficiencyWarning)
/usr/local/lib/python3.6/dist-packages/scipy/sparse/linalg/dsolve/linsolve.py:145: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  SparseEfficiencyWarning)
/usr/local/lib/python3.6/dist-packages/scipy/sparse/linalg/dsolve/linsolve.py:145: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  SparseEfficiencyWarning)
/usr/local/lib/python3.6/dist-packages/scipy/sparse/linalg/dsolve/linsolve.py:145: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  SparseEfficiencyWarning)
/usr/local/lib/python3.6/dist-packages/scipy/sparse/linalg/dsolve/linsolve.py:145: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  SparseEfficiencyWarning)
/usr/local/lib/python3.6/dist-packages/scipy/sparse/linalg/dsolve/linsolve.py:145: SparseE

In [66]:
unique_valid_posts.to_pickle('./results/argQ/unique_valid_results.pkl')

In [38]:
# haven't used it.
unique_valid_posts.to_pickle('/mnt/ceph/storage/data-in-progress/data-research/arguana/arg-generation/multi-taks-counter-argument-generation/reddit_data/conclusion_and_ca_generation/valid_conclusion_comp_remove_75sem_perc_unique_posts_with_arglexrank_conclusions.pkl')

In [60]:
unique_valid_posts[['post_id', 'title', 'post', 'arglex_rank_conclusion']]

,post_id,title,post,arglex_rank_conclusion
295914,t3_7uek1b,Pink is just light red and doesn't deserve it's own name.,"[i understand the need for more precise naming when you're doing something artsy., what i'm focusing on is how we describe colors in everyday life., pink is just a light shade of the color red., we already have a ton of precedence for just calling something a light or dark color., when people see a light or dark blue object, they just call it light blue or dark blue., when the see a light or dark green thing, they call it light green or dark green., for some reason, when we see something that is light red we call it pink., i don't see any rhyme or reason to this distinction and i don't thi...","[(when people see a light or dark blue object, they just call it light blue or dark blue., 0.12766313749127695), (we already have a ton of precedence for just calling something a light or dark color., 0.12701010687013486), (for some reason, when we see something that is light red we call it pink., 0.12668305146081252), (i don't see any rhyme or reason to this distinction and i don't think pink deserves it's name., 0.1264818820496689), (i understand the need for more precise naming when you're doing something artsy., 0.12512283443680042), (what i'm focusing on is how we describe colors in e..."
295939,t3_7uj99e,Elon Musk should not be selling flamethrowers as if they were toys.,"[i just saw on the news that elon musk have sold over 20 thousand flamethrowers., they appear to be oversized blowtorches designed to be just below the limits that would require licenses in most states., in a promotional video, he jokes around with it, pretending to use it on the cameraman. on twitter, he jokes about it been useful during a zombie apocalypse., but, as far as i know, no real use for it is seriously discussed., i find this behaviour borderline negligent., if these things become even more popular, it is just a matter of time until someone gets seriously hurt., either on purpo...","[(if these things become even more popular, it is just a matter of time until someone gets seriously hurt., 0.07612257015414309), (they appear to be oversized blowtorches designed to be just below the limits that would require licenses in most states., 0.07610165837093047), (a lot of comments point out that cars are also dangerous., 0.07456529619525219), (i find this behaviour borderline negligent., 0.07321782055938872), (while this might be true, the fact is that the product was not designed to be efficiently used as a blow torch., 0.07300115247393109), (lots of comments mention the fact ..."
295977,t3_7ujtee,"The Commandment ""Thou Shalt Not Kill"" is not hypocritical when reviewing the actions of Old Testament God; it's just poorly translated.","[this is kind of a weird one because i am an atheist and generally dislike a lot of religious dogma, but it is a view i hold after looking deeper into an issue i often hear in anti theistic and or atheistic discussions of christian hypocrisy., and while this may be appropriate for a religious sub like r debatereligion or something similar, cmv seems like it would have a wider audience., the title doesn't fully encompass my view, so i'll try to explain it better down below., background : the ten commandments are a list of moral imperatives that many adherents of judaism and christianity sti...","[(. christians who support these stances are often mocked for hypocris, 0.03245842184656077), (. and i imagine that it would be a shocking culture change to many jews and christians, as it would require a reorienting of their faith from 'life is sacred and so killing is bad' to 'killing is wrong only if it displeases god, 0.032380790551334845), (background : the ten commandments are a list of moral imperatives that many adherents of judaism and christianity still believe to be vitally important today., 0.032268332309299604), (. as wikipedia notes shalt not kill , this is often translated t..."
296048,t3_7ulxk7,"You can be

In [61]:
post_to_conc = pd.Series([ ranked_candidates[0][0] for ranked_candidates in  unique_valid_posts.arglex_rank_conclusion.tolist()], index=unique_valid_posts.post_id).to_dict()

In [62]:
valid_df['arglex_rank_conclusion'] = valid_df.post_id.apply(lambda x: post_to_conc[x])

In [67]:
valid_df.to_pickle('./results/argQ/valid_results.pkl')

In [26]:
# havent used it
valid_df.to_pickle('../../../data-ceph/arguana/arg-generation/multi-taks-counter-argument-generation/reddit_data/conclusion_and_ca_generation/valid_conclusion_comp_remove_75sem_perc_with_targets.pkl')